In [2]:
import sqlite3
import math
import sklearn
import re
import numpy as np
from scipy import spatial
from nltk.tokenize import regexp_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords 
from nltk.tokenize import regexp_tokenize
from nltk.stem import PorterStemmer
import pandas as pd

In [3]:
df = pd.read_csv(r'C:\Users\Antonio franco\Documents\IR\project\export.csv')
df.head()

,text,class
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [4]:
df['class'].unique()

array(['business', 'engineering', 'health', 'art', 'irrelevant'],
      dtype=object)

In [5]:
df['class'].value_counts()

art            101
business        99
engineering     93
irrelevant      93
health          88
Name: class, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

In [9]:
X = df['text']
y = df['class']

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.3,  random_state=42)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [36]:
vectorizer = TfidfVectorizer()
dataf = vectorizer.fit(X_train)
X_train_tfidf = dataf.transform(X_train)
X_test_tfidf = dataf.transform(X_test)

In [22]:
!pip install scikit-optimize
from skopt import BayesSearchCV
from sklearn.naive_bayes import MultinomialNB

  Using cached scikit_optimize-0.8.1-py2.py3-none-any.whl (101 kB)
  Using cached pyaml-20.4.0-py2.py3-none-any.whl (17 kB)


In [49]:
from sklearn import model_selection
from skopt.space import Real, Categorical, Integer
gNB = BayesSearchCV(
     MultinomialNB(),
     {
         'alpha': Real(1e-2, 1e+2, prior='log-uniform'),
     },
     n_iter=25,
     random_state=0
)

gNB.fit(X_train_tfidf, y_train)

C:\Users\Antonio franco\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\Antonio franco\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\Antonio franco\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


BayesSearchCV(estimator=MultinomialNB(), n_iter=25, random_state=0,
              search_spaces={'alpha': Real(low=0.01, high=100.0, prior='log-uniform', transform='identity')})

In [50]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
predictions = gNB.best_estimator_.predict(X_test_tfidf)

print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

         art       0.96      0.85      0.90        26
    business       0.93      0.93      0.93        41
 engineering       0.73      0.96      0.83        25
      health       0.86      1.00      0.93        19
  irrelevant       0.96      0.72      0.82        32

    accuracy                           0.88       143
   macro avg       0.89      0.89      0.88       143
weighted avg       0.90      0.88      0.88       143



In [53]:
from nltk.tokenize import word_tokenize
new_sentence = 'what is the price of the book'
new_word_list = word_tokenize(['what is the price of the book'])

TypeError: expected string or bytes-like object

In [55]:
text = dataf.transform(new_word_list)

In [57]:
predictions = gNB.best_estimator_.predict(['what is the price of the book'])

ValueError: Expected 2D array, got 1D array instead:
array=['what is the price of the book'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [41]:
predictions

array(['health', 'health', 'art', 'engineering', 'art', 'art', 'art'],
      dtype='<U11')

In [59]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf, y_train)

LinearSVC()

In [60]:
from sklearn.pipeline import Pipeline

In [61]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [62]:
text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [63]:
pred = text_clf.predict(X_test)

In [64]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics

print(metrics.classification_report(y_test, pred))

              precision    recall  f1-score   support

         art       1.00      0.96      0.98        26
    business       0.91      0.95      0.93        41
 engineering       0.96      1.00      0.98        25
      health       0.79      1.00      0.88        19
  irrelevant       0.96      0.75      0.84        32

    accuracy                           0.92       143
   macro avg       0.92      0.93      0.92       143
weighted avg       0.93      0.92      0.92       143



In [79]:
text_clf.predict(['talking a  walk to muse'])

array(['health'], dtype=object)

In [69]:
import pickle

# save the model to disk
pickle.dump(text_clf, open('modelText', 'wb'))

In [70]:
modelText = pickle.load(open('C:/Users/Antonio franco/Documents/IR/project/modelText', 'rb'))


In [73]:
s ='my heart is yours my love'

In [75]:
n =[]
n.append(s)


In [76]:
modelText.predict(n)[0]


'health'

In [34]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
nb_model = MultinomialNB()
nb_model.fit(X, y_train)

predictions = nb_model.predict(X_test)

print(metrics.classification_report(y_test, predictions))

ValueError: could not convert string to float: 'Verdict delay for Greek sprinters\n\nGreek athletics\' governing body has postponed by two weeks the judgement on sprinters Costas Kenteris and Katerina Thanou for missing doping tests.\n\nThe pair are facing lengthy bans for the missed tests, including one on the eve of last year\'s Athens Olympics. They were set to learn their fate by the end of February, but late evidence from them has pushed the date back. "A decision is now expected by around mid-March," said one of their lawyers, Michalis Dimitrakopoulos. Kenteris, 31, who won the men\'s 200m title at the 2000 Sydney Games and Thanou, 30, who won the women\'s 100m silver medal in Sydney, face a maximum two-year ban if found guilty. The athletes, who spectacularly withdrew from the Athens Olympics, have been suspended by the International Association of Athletics Federations (IAAF) for missing the three tests. The IAAF said the sprinters had failed to provide samples for tests in Tel

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
count_vect = CountVectorizer()

In [16]:
X_train_counts = count_vect.fit_transform(X_train)

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer

In [12]:
tdidf_transformer = TfidfTransformer()

In [17]:
X_train_tfidf = tdidf_transformer.fit_transform(X_train_counts)

In [18]:
X_train_tfidf.shape

(331, 13929)